In [0]:
import sys
sys.path.insert(0, "../lib/")
import ingestores


# tabela = dbutils.widgets.get("tabela")
tabela ="cotacoes"
schema = "cotacoes_bronze"
catalog = "prod"
checkpointpath = f"/Volumes/transacional/cotacoes_db/checkpoint/checkpoint_{tabela}"
dbpath = f"transacional.cotacoes_db.{tabela}"


if not spark.catalog.tableExists(f"{catalog}.{schema}.{tabela}"):

        print("Tabela não existente, criando.")
        dbutils.fs.rm(checkpointpath, recurse=True)  
        ingestao_full_load = ingestores.ingestao(spark, tabela, schema, catalog, dbpath)
        
        if ingestao_full_load.run():
                print("Tabela criada com sucesso!")
        else:
                print("Falha ao criar tabela!")

else:
        print("Tabela já existente, ignorando FULL LOAD")
        ingestao_cdc = ingestores.ingestaoCDC(spark, tabela, schema, catalog, dbpath, checkpointpath)
        ingestao_cdc.runCDC()